In [ ]:
!pip install langchain==0.1.14
!pip install langchain-experimental==0.0.56
!pip install langchain-community==0.0.31
!pip install faiss-cpu==1.8.0
!pip install pdfplumber==0.11.0
!pip install gradio==4.25.0

In [11]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("/Users/kailashdejesushornig/Documents/GitHub/P2_Policydokument/data/all_files/Alkohol- och drogpolicy.PDF")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

# Load the random page content
docs[0].page_content

# Output
"""
Leaf Diseases Caused By Fungi and Bacteria
Leaf Spots
Bacteria or fungi can cause leaf spots that vary in size, shape, and color. Usually the spot has a distinct margin and may be surrounded by a yellow
halo. A fungal leaf spot nearly always has a growth of some type in the spot, particularly in damp weather. It may be a tiny pimple-like structure or a
moldy growth of spores. Often the structures are visible through a hand lens. Nearby diseased areas may join to form irregular "blotches."
...
"""

Number of pages in the PDF: 1


'\nLeaf Diseases Caused By Fungi and Bacteria\nLeaf Spots\nBacteria or fungi can cause leaf spots that vary in size, shape, and color. Usually the spot has a distinct margin and may be surrounded by a yellow\nhalo. A fungal leaf spot nearly always has a growth of some type in the spot, particularly in damp weather. It may be a tiny pimple-like structure or a\nmoldy growth of spores. Often the structures are visible through a hand lens. Nearby diseased areas may join to form irregular "blotches."\nphoto: Paul Bachi, University of Kentucky Research and Education Center, Bugwood.org\nSeptoria brown spot is a common fungal disease of soybeans. It causes small angular red-brown spots to develop on upper and lower surfaces of\ntrifoliate leaves 2 to 3 weeks after planting. Numerous spots will cause leaves to yellow and drop. The disease will develop many irregular, tan lesions\non trifoliate leaves that later turn dark brown. Individual spots will frequently coalesce to form large blackish-b

In [9]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

# Check number of chunks created
print("Number of chunks created: ", len(documents))
# Output 
"""
Number of chunks created:  2
"""

# Printing first few chunks
for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

# Output
"""
CHUNK : 1
Besluta v: Typa v styrdokument: Diarienummer:
Rektor Policy C 2023-1732
Datum för beslu:t Handläggar: e
See -signering Katrin Axelsson
Dokumentetg ällerf rån Avdelning/motsvarandseo ma nsvararf ör att dokumentets kapaso ch/eller
ochm ed revideras:
2024-03-18 HR-avdelningen
Dokumentetg ällert ill Dokumentete rsättert idigareb eslut:
ochm ed: C 2008/538 - Policyo ch föreskrift avseendea lkohol och droger vid Chalmers. Tills vidare Föreliggandep olicy ersätter policy-deleni tidigare beslut.

"""

/Users/kailashdejesushornig/Documents/GitHub/P2_Policydokument/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Number of chunks created:  2

CHUNK : 1
Besluta v: Typa v styrdokument: Diarienummer:
Rektor Policy C 2023-1732
Datum för beslu:t Handläggar: e
See -signering Katrin Axelsson
Dokumentetg ällerf rån Avdelning/motsvarandseo ma nsvararf ör att dokumentets kapaso ch/eller
ochm ed revideras:
2024-03-18 HR-avdelningen
Dokumentetg ällert ill Dokumentete rsättert idigareb eslut:
ochm ed: C 2008/538 - Policyo ch föreskrift avseendea lkohol och droger vid Chalmers. Tills vidare Föreliggandep olicy ersätter policy-deleni tidigare beslut.

CHUNK : 2
Föreskrift gäller fram tills
ny versiona v föreskrift beslutas. Policya vseendea lkohol-ochd roger
Styrdokumenvt id Chalmerste kniskah ögskolaA B
En alkohol- och drogfri högskolaä r en förutsättning för en god arbets- och studiemiljö. Alkoholo ch
droger hör inte ihop med arbete och studier. På Chalmersä r konsumtiona v alkohol eller
drogmissbruku nder arbets-e ller studietid inte tillåten och eventuellt bruk på fritiden får inte påverka
säkerhet,a rbet

'\n\nCHUNK : 1\nKentucky Pesticide Education Program\ncopyright © 2016 University of Kentucky Department of Entomology\nAgricultural Plant Diseases\nPlant Diseases\nA plant disease is any harmful condition that affects a plant’s appearance or function. Common pathogens that cause diseases include: fungi,\nbacteria, and viruses. Some nematodes are plant disease agents. Temperature extremes or nutrient deficiencies are examples of disorders caused by\nnon-infectious factors. The disease triangle is a fundamental concept in plant pathology. Disease occurs only when all three sides of the triangle are present: a susceptible\nhost, apathogen (the agent that causes disease), and an environment favorable for disease to develop. Plant diseases are managed by manipulating\nthe disease triangle: the plant, the pathogen, and/or the environment. Infection begins when the pathogen enters the plant. The disease process starts when it arrives at a part of a plant where infection can occur. If\nenviro

In [12]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
import gradio as gr

# Load the PDF
loader = PDFPlumberLoader("/Users/kailashdejesushornig/Documents/GitHub/P2_Policydokument/data/all_files/Alkohol- och drogpolicy.PDF")
docs = loader.load()

# Split into chunks
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)


# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store and fill it with embeddings
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define llm
llm = Ollama(model="mistral")

# Define the prompt
prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None)
              
qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True)

def respond(question,history):
    return qa(question)["result"]


gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Ask me question related to Plants and their diseases", container=False, scale=7),
    title="Plant's Chatbot",
    examples=["What are different kinds of plant diseases", "What is Stewart’s wilt disease"],
    cache_examples=True,
    retry_btn=None,

).launch(share = True)

Caching examples at: '/Users/kailashdejesushornig/Documents/GitHub/P2_Policydokument/script/gradio_cached_examples/14'
Caching example 1/2


/Users/kailashdejesushornig/Documents/GitHub/P2_Policydokument/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...
IMPORTANT: You are using gradio version 4.25.0, however version 4.29.0 is available, please upgrade.
--------


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:Föreskrift gäller fram tills
ny versiona v föreskrift beslutas. Policya vseendea lkohol-ochd roger
Styrdokumenvt id Chalmerste kniskah ögskolaA B
En alkohol- och drogfri högskolaä r en förutsättning för en god arbets- och studiemiljö. Alkoholo ch
droger hör inte ihop med arbete och studier. På Chalmersä r konsumtiona v alkohol eller
drogmissbruku nder arbets-e ller studietid inte tillåten och eventuellt bruk på fritiden får inte påverka
säkerhet,a rbetsförmågae ller trivsel på arbetsplatsen. Alla som arbetar eller studerar vi

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull mistral`.